In [3]:
# 1. Install necessary libraries
# 1. Install necessary libraries
!pip install pandas pydantic google-genai requests

import pandas as pd
import json
import os
import requests # To fetch TikTok video context
from typing import List, Dict, Optional
from pydantic import BaseModel, Field # For structured output schema
import time # For API call rate limiting
from google import genai # Using this import as shown in your traceback

# --- LLM Response Schema (CRUCIAL for structured output) ---
class CommentClassification(BaseModel):
    """Schema for a single comment's classification result."""
    comment_id: str = Field(..., description="The original comment_id from the input.")
    classification: str = Field(..., description="The classification: 'HATE_SPEECH' or 'NOT_HATE_SPEECH'.")
    confidence: float = Field(..., description="A confidence score from 0.0 to 1.0.")
    justification: str = Field(..., description="A brief Vietnamese reason for the decision, citing the comment or video context.")

class ClassificationList(BaseModel):
    """The full list of classification results returned by the LLM."""
    results: List[CommentClassification]

# --- LLM API Setup (CORRECTED) ---
# NOTE: This code is designed to run in Google Colab.
try:
    from google.colab import userdata

    # 1. Click the "Key" icon (🔑) on the left panel of Colab.
    # 2. Add a new secret.
    # 3. Name the secret: GEMINI_API_KEY
    # 4. Paste your API key as the value.

    API_KEY = userdata.get('GEMINI_API_KEY')
    if not API_KEY:
        raise ValueError("API key not found in Colab Secrets. Please add it.")

    # Pass the API key DIRECTLY to the Client constructor
    client = genai.Client(api_key=API_KEY)

    MODEL_NAME = "gemini-2.5-flash"
    print("✅ Gemini Client successfully initialized with API key.")

except ImportError:
    print("Not in Colab. Attempting to fall back to environment variables...")
    API_KEY = os.getenv("GEMINI_API_KEY")
    if not API_KEY:
        print("!!! ERROR: GEMINI_API_KEY environment variable not set.")
    else:
        # Pass the API key DIRECTLY to the Client constructor
        client = genai.Client(api_key=API_KEY)
        MODEL_NAME = "gemini-2.5-flash"
        print("✅ Gemini Client successfully configured from environment variable.")

except Exception as e:
    print(f"!!! Error initializing Gemini Client: {e}")


from google.colab import drive
drive.mount('/content/drive')

print("Setup complete. Libraries installed and imports ready.")

✅ Gemini Client successfully initialized with API key.
Mounted at /content/drive
Setup complete. Libraries installed and imports ready.


In [5]:
%cd /content/drive/MyDrive/Preprocessed_Comments

/content/drive/MyDrive/Preprocessed_Comments


In [9]:
# --- TikTok Context Fetching (Using Simplified, Robust Pattern) ---

VIDEO_CONTEXT_CACHE = {}

def get_video_context(video_id: str, retries: int = 3, delay: float = 0.5) -> Optional[str]:
    """
    Fetches the video caption/hashtags using the simple API endpoint.
    This is based on your provided, more stable function.
    """
    if video_id in VIDEO_CONTEXT_CACHE:
        return VIDEO_CONTEXT_CACHE[video_id]

    # Using the simplified endpoint and params you provided
    API_ENDPOINT = "https://www.tiktok.com/api/comment/list/"
    # We only need 1 comment to get the share_info
    PARAMS = {"aid": "1988", "aweme_id": video_id, "count": 1, "cursor": 0}
    HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

    # --- Retry and Delay Logic ---
    for attempt in range(retries):
        try:
            # Add delay BEFORE the request to respect the API endpoint
            print(f"  > Attempt {attempt+1}/{retries} for {video_id}. Waiting {delay}s...")
            time.sleep(delay)

            response = requests.get(API_ENDPOINT, params=PARAMS, headers=HEADERS, timeout=10)
            response.raise_for_status() # Raise an exception for 4xx/5xx status codes
            data = response.json()

            if 'comments' in data and data['comments']:
                # Extract context from the 'share_info' of the first comment
                title = data['comments'][0]['share_info']['title']
                VIDEO_CONTEXT_CACHE[video_id] = title.strip()
                return title.strip()

            # If 'comments' key exists but is empty
            VIDEO_CONTEXT_CACHE[video_id] = "Context Not Found (Empty Comments)."
            return "Context Not Found (Empty Comments)."

        except requests.exceptions.RequestException as e:
            print(f"  > TikTok API Error (Attempt {attempt+1}): {e}")
            if attempt == retries - 1: # Last attempt failed
                VIDEO_CONTEXT_CACHE[video_id] = "API Failure."
                return "API Failure."
            # Wait longer before retrying
            time.sleep(1) # Extra 1s wait on failure

    # Fallback in case loop finishes unexpectedly
    return "API Failure."


## Function to Load Data (Using 4-digit zero-padding)
def load_and_merge_comments(file_prefix: str = 'clean_comments_batch_', num_files: int = 5) -> pd.DataFrame:
    """Loads a few comment files and merges them into a single DataFrame."""
    all_data = []
    print(f"Loading files with prefix: {file_prefix}XXXX.csv")
    for i in range(1, num_files + 1):
        try:
            # Use 4-digit zero-padding (e.g., 0001, 0002)
            file_name = f"{file_prefix}{i:04d}.csv"
            df = pd.read_csv(file_name)
            all_data.append(df)
            print(f"Loaded {file_name}")
        except FileNotFoundError:
            print(f"File not found: {file_name}. Stopping data loading.")
            break

    if all_data:
        merged_df = pd.concat(all_data, ignore_index=True)
        merged_df['video_id'] = merged_df['video_id'].astype(str) # 'aweme_id'
        return merged_df.drop_duplicates(subset=['comment_id'])
    return pd.DataFrame()

## Load Data (change num_files as needed)
df_comments = load_and_merge_comments(file_prefix='clean_comments_batch_', num_files=3)
print(f"\nTotal unique comments loaded: {len(df_comments)}")

## Add Context to Comments and Create JSON Payload (Using 'clean_text')
def enrich_comments_with_context(df: pd.DataFrame) -> pd.DataFrame:
    """Fetches video context and creates the structured payload for the prompt."""
    enriched_data = []

    unique_video_ids = df['video_id'].unique()
    print(f"\nFetching context for {len(unique_video_ids)} unique videos...")

    for i, vid in enumerate(unique_video_ids):
        print(f"Processing Video {i+1}/{len(unique_video_ids)} (ID: {vid})")
        # The delay and retry logic is handled inside get_video_context()
        get_video_context(vid) # Populates the cache

    print("\nContext fetching complete. Starting comment structurization...")

    for _, row in df.iterrows():
        vid = row['video_id']
        video_context_combined = VIDEO_CONTEXT_CACHE.get(vid, "Context Not Found.")

        # Check if 'clean_text' column exists, fallback to 'comment_text' if not
        comment_content = row['clean_text'] if 'clean_text' in row and pd.notna(row['clean_text']) else row['comment_text']

        comment_data = {
            "comment_id": str(row['comment_id']),
            "video_context": video_context_combined,
            "commenter_username": row['username'],
            # --- Using 'clean_text' (with fallback) ---
            "comment_content": comment_content,
        }

        row_with_context = row.to_dict()
        row_with_context['video_context'] = video_context_combined
        row_with_context['comment_json'] = json.dumps(comment_data, ensure_ascii=False)
        enriched_data.append(row_with_context)

    return pd.DataFrame(enriched_data)

df_enriched = enrich_comments_with_context(df_comments)
if not df_enriched.empty:
    print(f"\nExample of structured object with context:\n{df_enriched['comment_json'].iloc[0]}")
else:
    print("\nNo data loaded or enriched. Check CSV file names and paths.")

Loading files with prefix: clean_comments_batch_XXXX.csv
Loaded clean_comments_batch_0001.csv
Loaded clean_comments_batch_0002.csv
Loaded clean_comments_batch_0003.csv

Total unique comments loaded: 3000

Fetching context for 58 unique videos...
Processing Video 1/58 (ID: 6989137370817842459)
  > Attempt 1/3 for 6989137370817842459. Waiting 0.5s...
Processing Video 2/58 (ID: 6994046144053660930)
  > Attempt 1/3 for 6994046144053660930. Waiting 0.5s...
Processing Video 3/58 (ID: 7013362152782171419)
  > Attempt 1/3 for 7013362152782171419. Waiting 0.5s...
Processing Video 4/58 (ID: 7040406003837373698)
  > Attempt 1/3 for 7040406003837373698. Waiting 0.5s...
Processing Video 5/58 (ID: 7041523740936572162)
  > Attempt 1/3 for 7041523740936572162. Waiting 0.5s...
Processing Video 6/58 (ID: 7044415240049528065)
  > Attempt 1/3 for 7044415240049528065. Waiting 0.5s...
Processing Video 7/58 (ID: 7067497026837105946)
  > Attempt 1/3 for 7067497026837105946. Waiting 0.5s...
Processing Video 8/

In [10]:
## Actual LLM API Call with Structured Output
def call_gemini_api(prompt: str) -> List[Dict]:
    """Calls the Gemini API with structured output configuration."""
    global client

    SYSTEM_INSTRUCTION = """
    You are an expert Vietnamese content moderator. Your task is to classify TikTok comments as hate speech or not.
    Hate speech is defined as any comment that **insults, degrades, or attacks any individual or social group**.

    Classification Rules:
    1. **Context is Key:** Use the 'video_context' to understand the comment's intent.
    2. **Hate Speech:** Classify as **'HATE_SPEECH'** if the comment contains insults, degradation, or attacks.
    3. **Ambiguity:** If the comment contains Vietnamese slang, inside jokes, or is out of context, and you cannot confidently classify it, assign a **LOW confidence score (e.g., 0.1 to 0.4)**.
    4. **Output:** Your entire response MUST be a single JSON object that strictly adheres to the provided schema (a list of results).
    """

    generation_config = genai.types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=ClassificationList,
        system_instruction=SYSTEM_INSTRUCTION
    )

    contents = [
        {"role": "user", "parts": [{"text": prompt}]}
    ]

    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=contents,
            config=generation_config
        )
        response_json = json.loads(response.text)
        return response_json.get('results', [])

    except Exception as e:
        print(f"!!! API/JSON Error during call: {e}")
        return []

## Prompt Generation and Batching Experiment
BATCH_SIZES = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
MAX_COMMENTS = sum(BATCH_SIZES)

df_test = df_enriched.head(MAX_COMMENTS).copy()
print(f"Using {len(df_test)} comments for the batching experiment.")

experiment_results = []
start_index = 0

print("\n--- Starting LLM Batching Experiment ---")

if df_test.empty:
    print("No data to test. Stopping experiment.")
else:
    for batch_size in BATCH_SIZES:
        print(f"\nProcessing batch size: {batch_size}...")

        batch_data = df_test.iloc[start_index:start_index + batch_size]
        if batch_data.empty:
            print("No more comments left. Stopping.")
            break

        comment_json_list = ',\n'.join(batch_data['comment_json'].tolist())

        user_prompt = f"""
        Please classify the following {len(batch_data)} TikTok comments based on the provided rules.

        Input Comments (List of JSON Objects):
        ---START_OF_INPUT---
        [
        {comment_json_list}
        ]
        ---END_OF_INPUT---
        """

        current_batch_ids = batch_data['comment_id'].tolist()

        llm_results = call_gemini_api(user_prompt)

        if len(llm_results) != len(current_batch_ids):
             print(f"!!! WARNING: LLM returned {len(llm_results)} results, expected {len(current_batch_ids)}. This batch size may be unstable.")

        for result in llm_results:
            result['batch_size_tested'] = batch_size

        experiment_results.extend(llm_results)

        start_index += batch_size
        # Add a small delay between batches
        time.sleep(1)

print("\n--- LLM Experiment Complete ---")
print(f"Total classification results collected: {len(experiment_results)}")

Using 1023 comments for the batching experiment.

--- Starting LLM Batching Experiment ---

Processing batch size: 1...

Processing batch size: 2...

Processing batch size: 4...

Processing batch size: 8...

Processing batch size: 16...

Processing batch size: 32...

Processing batch size: 64...

Processing batch size: 128...

Processing batch size: 256...

Processing batch size: 512...
!!! API/JSON Error during call: Expecting value: line 184 column 20 (char 7189)
!!! WARNING: LLM returned 0 results, expected 512. This batch size may be unstable.

--- LLM Experiment Complete ---
Total classification results collected: 511


In [11]:
## Process and Merge Results
if not experiment_results:
    print("No results were collected from the LLM. Cannot save files.")
else:
    df_results = pd.DataFrame(experiment_results)

    df_results = df_results.rename(columns={
        'classification': 'llm_hate_speech',
        'confidence': 'llm_confidence',
        'justification': 'llm_justification'
    })

    df_results['comment_id'] = df_results['comment_id'].astype(str)
    df_enriched['comment_id'] = df_enriched['comment_id'].astype(str)

    df_final = pd.merge(
        df_enriched.drop(columns=['comment_json']),
        df_results,
        on='comment_id',
        how='left'
    )

    print("\nFinal DataFrame structure:")
    print(df_final.columns.tolist())

    ## Save Results by Batch Size
    print("\n--- Saving Results ---")
    for batch_size in BATCH_SIZES:
        df_batch = df_final[df_final['batch_size_tested'] == batch_size].copy()

        if not df_batch.empty:
            cols_to_keep = [
                'comment_id', 'video_id', 'username', 'comment_text',
                'digg_count', 'create_time', 'clean_text', 'video_context',
                'llm_hate_speech', 'llm_confidence', 'llm_justification',
                'batch_size_tested'
            ]

            df_batch = df_batch[[col for col in cols_to_keep if col in df_batch.columns]]

            output_filename = f"llm_classification_batch_{batch_size}.csv"
            df_batch.to_csv(output_filename, index=False)
            print(f"Saved results for batch size {batch_size} to: {output_filename} ({len(df_batch)} rows)")

    print("\nAll tasks complete. Review the CSV files to determine the optimal batch size.")


Final DataFrame structure:
['comment_id', 'video_id', 'username', 'comment_text', 'digg_count', 'create_time', 'clean_text', 'video_context', 'llm_hate_speech', 'llm_confidence', 'llm_justification', 'batch_size_tested']

--- Saving Results ---
Saved results for batch size 1 to: llm_classification_batch_1.csv (1 rows)
Saved results for batch size 2 to: llm_classification_batch_2.csv (2 rows)
Saved results for batch size 4 to: llm_classification_batch_4.csv (4 rows)
Saved results for batch size 8 to: llm_classification_batch_8.csv (8 rows)
Saved results for batch size 16 to: llm_classification_batch_16.csv (16 rows)
Saved results for batch size 32 to: llm_classification_batch_32.csv (32 rows)
Saved results for batch size 64 to: llm_classification_batch_64.csv (64 rows)
Saved results for batch size 128 to: llm_classification_batch_128.csv (128 rows)
Saved results for batch size 256 to: llm_classification_batch_256.csv (256 rows)

All tasks complete. Review the CSV files to determine the

In [13]:
import math

# --- Constants for the Forecast ---
BATCH_SIZE = 256
NUM_BATCHES_TO_TEST = 10
NUM_FILES_TO_LOAD = 3 # Just enough to get ~3000 comments
TOTAL_PROJECT_COMMENTS = 131000 # Use your known total for the final forecast

token_counts = []

print("--- Starting Billing Forecast (Optimized) ---")
print(f"Loading {NUM_FILES_TO_LOAD} files for a sample...")

# 1. Load a sample of comments (using the function from Cell 2)
# This re-uses 'load_and_merge_comments' from Cell 2
df_sample_comments = load_and_merge_comments(
    file_prefix='clean_comments_batch_',
    num_files=NUM_FILES_TO_LOAD
)

if not df_sample_comments.empty and len(df_sample_comments) >= (BATCH_SIZE * NUM_BATCHES_TO_TEST):

    # 2. Enrich only the sample comments
    # This re-uses 'enrich_comments_with_context' from Cell 2
    print("\nEnriching sample comments with video context. This will take a moment...")
    df_enriched_sample = enrich_comments_with_context(df_sample_comments)

    # 3. Re-create the System Instruction from Cell 3
    SYSTEM_INSTRUCTION = """
    You are an expert Vietnamese content moderator. Your task is to classify TikTok comments as hate speech or not.
    Hate speech is defined as any comment that **insults, degrades, or attacks any individual or social group**.

    Classification Rules:
    1. **Context is Key:** Use the 'video_context' to understand the comment's intent.
    2. **Hate Speech:** Classify as **'HATE_SPEECH'** if the comment contains insults, degradation, or attacks.
    3. **Ambiguity:** If the comment contains Vietnamese slang, inside jokes, or is out of context, and you cannot confidently classify it, assign a **LOW confidence score (e.g., 0.1 to 0.4)**.
    4. **Output:** Your entire response MUST be a single JSON object that strictly adheres to the provided schema (a list of results).
    """

    print(f"\nCounting tokens for {NUM_BATCHES_TO_TEST} batches of {BATCH_SIZE} comments...")
    start_index = 0

    for i in range(NUM_BATCHES_TO_TEST):
        end_index = start_index + BATCH_SIZE
        batch_data = df_enriched_sample.iloc[start_index:end_index]

        # Get the JSON payload for the batch
        comment_json_list = ',\n'.join(batch_data['comment_json'].tolist())

        # Re-create the user prompt
        user_prompt = f"""
        Please classify the following {len(batch_data)} TikTok comments based on the provided rules.

        Input Comments (List of JSON Objects):
        ---START_OF_INPUT---
        [
        {comment_json_list}
        ]
        ---END_OF_INPUT---
        """

        # Create the full 'contents' object for the API
        contents = [
            {"role": "user", "parts": [
                {"text": SYSTEM_INSTRUCTION},
                {"text": user_prompt}
            ]}
        ]

        try:
            # 4. Call the count_tokens API
            response = client.models.count_tokens(model=MODEL_NAME, contents=contents)
            token_count = response.total_tokens
            token_counts.append(token_count)
            print(f"  > Batch {i+1} token count: {token_count:,}")

        except Exception as e:
            print(f"  > Error counting tokens for batch {i+1}: {e}")

        start_index = end_index

    # 5. Calculate and Report the Forecast
    if token_counts:
        total_test_tokens = sum(token_counts)
        avg_tokens_per_batch = total_test_tokens / len(token_counts)

        # Use math.ceil to ensure we process all comments
        total_batches_needed = math.ceil(TOTAL_PROJECT_COMMENTS / BATCH_SIZE)

        total_estimated_tokens = total_batches_needed * avg_tokens_per_batch

        print("\n--- Token Analysis (10 Batches) ---")
        print(f"Average INPUT tokens per {BATCH_SIZE}-comment batch: {avg_tokens_per_batch:,.0f}")

        print("\n--- Total Project Forecast ({TOTAL_PROJECT_COMMENTS:,} Comments) ---")
        print(f"Total batches needed (at {BATCH_SIZE}/batch): {total_batches_needed:,}")
        print(f"Total estimated INPUT tokens for all comments: {total_estimated_tokens:,.0f}")

        print("\n--- Billing Forecast ---")
        print("To calculate your cost, use this formula:")
        print("  (Total Estimated Tokens / 1,000,000) * $PRICE_PER_1M_INPUT_TOKENS")
        print(f"\nExample using '{MODEL_NAME}' pricing ($0.08 per 1M input tokens, <128k context):")

        # Example calculation (price is an assumption)
        example_price_per_1m = 0.08
        example_cost = (total_estimated_tokens / 1_000_000) * example_price_per_1m

        print(f"  ({total_estimated_tokens:,.0f} / 1,000,000) * ${example_price_per_1m} = ${example_cost:.4f}")

        print("\n*Note: This forecast is for INPUT tokens only. Output tokens are billed separately but are usually much smaller for classification tasks.")

    else:
        print("No token counts were calculated. Check data loading and API connection.")

else:
    print(f"Failed to load enough data. Needed {BATCH_SIZE * NUM_BATCHES_TO_TEST} comments but only loaded {len(df_sample_comments)}. Check file names or increase 'NUM_FILES_TO_LOAD'.")

--- Starting Billing Forecast (Optimized) ---
Loading 3 files for a sample...
Loading files with prefix: clean_comments_batch_XXXX.csv
Loaded clean_comments_batch_0001.csv
Loaded clean_comments_batch_0002.csv
Loaded clean_comments_batch_0003.csv

Enriching sample comments with video context. This will take a moment...

Fetching context for 58 unique videos...
Processing Video 1/58 (ID: 6989137370817842459)
Processing Video 2/58 (ID: 6994046144053660930)
Processing Video 3/58 (ID: 7013362152782171419)
Processing Video 4/58 (ID: 7040406003837373698)
Processing Video 5/58 (ID: 7041523740936572162)
Processing Video 6/58 (ID: 7044415240049528065)
Processing Video 7/58 (ID: 7067497026837105946)
Processing Video 8/58 (ID: 7079327904911412507)
Processing Video 9/58 (ID: 7121668783567113498)
Processing Video 10/58 (ID: 7122090630163041563)
Processing Video 11/58 (ID: 7139855079187778843)
Processing Video 12/58 (ID: 7145739858370120987)
Processing Video 13/58 (ID: 7151196519641763098)
Processing